<a href="https://colab.research.google.com/github/camenduru/notebooks/blob/main/camenduru's_stable_diffusion_discord.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!pip install -qq git+https://github.com/camenduru/diffusers.git
!pip install -qq transformers ftfy

In [2]:
import torch, os, gc, requests, random
from diffusers import StableDiffusionPipeline
from torch import autocast

In [4]:
def patch_conv(cls):
	init = cls.__init__
	def __init__(self, *args, **kwargs):
		return init(self, *args, **kwargs, padding_mode='circular')
	cls.__init__ = __init__

patch_conv(torch.nn.Conv2d)

In [3]:
pipe = StableDiffusionPipeline.from_pretrained("/content/gdrive/MyDrive/AI/StableDiffusion/models/stable-diffusion-v1-4-fp16", revision="fp16", torch_dtype=torch.float16).to("cuda")
pipe.safety_checker = lambda images, **kwargs: [images, [False] * len(images)]

In [ ]:
token = 'Bot TAyMDM1NTQ2MzI4MTM3NzM5MA.GRCh6J.N_6KM35TKuvMNZvrkoIXOjmZk3zHqDArpUoz6Q'
channel_id = 1020517233757261896
header = {"authorization": token}

from PIL.PngImagePlugin import PngInfo
metadata = PngInfo()

height = 512
width = 512
prompt = "cat"
metadata.add_text("Prompt", f"{prompt}")
metadata.add_text("by", "camenduru")

while True:
    gc.collect()
    torch.cuda.empty_cache()
    with autocast("cuda"):
      image = pipe(prompt, height=height, width=width, num_inference_steps=50, eta=0.0, guidance_scale=7.5)["sample"][0]

    random_name = random.randint(0,10000000)
    image.save(f"{random_name}.png", pnginfo=metadata)
    files = {f"{random_name}.png" : open(f"{random_name}.png", "rb").read()}
    payload = {"content":f"{prompt} by camenduru"}
    r = requests.post(f"https://discord.com/api/v9/channels/{channel_id}/messages", data=payload, headers=header, files=files).text